In [1]:
import sys, os
sys.path.insert(0, os.path.dirname(os.path.abspath('../desdeo_emo')))

In [2]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd

from desdeo_problem.problem.Variable import variable_builder
from desdeo_problem.problem.Objective import _ScalarObjective
from desdeo_problem.problem import MOProblem
from desdeo_problem.testproblems.TestProblems import test_problem_builder
from pygmo import fast_non_dominated_sorting as nds
from desdeo_emo.utilities.ReferenceVectors import ReferenceVectors



from desdeo_emo.EAs.MOEAD_NUMS import MOEA_D_NUMS
from desdeo_emo.EAs.MOEAD_NUMS_PLUS import MOEA_D_NUMS_PLUS
from desdeo_emo.utilities.plotlyanimate import animate_init_, animate_next_
from sklearn.preprocessing import Normalizer

from pymoo.indicators.rmetric import RMetric

from pymoo.factory import get_problem, get_reference_directions

In [3]:
import baseADM
from baseADM import *
import generatePreference as gp

## Create the problem object

In [4]:
n_obj = 3
n_var = 7
problem_name = "DTLZ1"
problem = test_problem_builder(name="DTLZ1", n_of_variables=7, n_of_objectives=3)
problem.ideal = np.asarray([0] * n_obj)
problem.nadir = np.asarray([1] * n_obj)

true_nadir = np.asarray([1] * n_obj)

## Using the EAs

Pass the problem object to the EA, pass parameters as arguments if required.

In [5]:
int_nums = MOEA_D_NUMS(problem=problem, interact=True, n_gen_per_iter=100, roi_size=0.7)
int_nums_plus = MOEA_D_NUMS_PLUS(problem=problem, interact=True, n_gen_per_iter=100, roi_size=0.7, num_solutions_display=5)

int_nums.start()
int_nums_plus.start()

In [6]:
reference_point = np.random.rand(n_obj)

pref_int_nums, plot_int_nums = int_nums.requests()
pref_int_nums_plus, plot_int_nums_plus = int_nums_plus.requests()

pref_int_nums[2].response = pd.DataFrame([reference_point], columns=pref_int_nums[2].content['dimensions_data'].columns)
pref_int_nums_plus[2].response = pd.DataFrame([reference_point], columns=pref_int_nums_plus[2].content['dimensions_data'].columns)


In [7]:
pref_int_nums, plot_int_nums  = int_nums.iterate(pref_int_nums[2])
pref_int_nums_plus, plot_int_nums_plus  = int_nums_plus.iterate(pref_int_nums_plus[2])

## Extracting optimized decision variables and objective values

In [8]:
objectives = int_nums.population.objectives
objectives2 = int_nums_plus.population.objectives


In [9]:
fig2 = go.Figure(data=go.Scatter3d(x=objectives[:,0],
                                   y=objectives[:,1],
                                   z=objectives[:,2],
                                   mode="markers",
                                   marker_size=2))
fig2

In [10]:
fig2 = go.Figure(data=go.Scatter3d(x=objectives2[:,0],
                                   y=objectives2[:,1],
                                   z=objectives2[:,2],
                                   mode="markers",
                                   marker_size=2))
fig2

In [11]:
cf = generate_composite_front(objectives, objectives2)
fig2 = go.Figure(data=go.Scatter3d(x=cf[:,0],
                                   y=cf[:,1],
                                   z=cf[:,2],
                                   mode="markers",
                                   marker_size=2))
fig2

In [12]:
# the following two lines for getting pareto front by using pymoo framework
problemR = get_problem(problem_name.lower(), n_var, n_obj)
ref_dirs = get_reference_directions("das-dennis", n_obj, n_partitions=12)
pareto_front = problemR.pareto_front(ref_dirs)



In [13]:
L = 4
D = 2
lattice_resolution = 5

# creates uniformly distributed reference vectors
reference_vectors = ReferenceVectors(lattice_resolution, n_obj)

In [15]:
for i in range(L):
    base = baseADM(cf, reference_vectors)
    response = gp.generateRP4learning(base)
    # run algorithms with the new reference point
    pref_int_nums[2].response = pd.DataFrame(
        [response], columns=pref_int_nums[2].content["dimensions_data"].columns
    )
    pref_int_nums_plus[2].response = pd.DataFrame(
        [response], columns=pref_int_nums_plus[2].content["dimensions_data"].columns)
    
    pref_int_nums, plot_int_nums = int_nums.iterate(pref_int_nums[2])
    pref_int_nums_plus, plot_int_nums_plus = int_nums_plus.iterate(pref_int_nums_plus[2])

    # extend composite front with newly obtained solutions
    cf = generate_composite_front(
        cf, int_nums.population.objectives, int_nums_plus.population.objectives)
    
    # R-metric calculation
    ref_point = response.reshape(1, n_obj)
    # normalize reference point
    rp_transformer = Normalizer().fit(ref_point)
    norm_rp = rp_transformer.transform(ref_point)
    rmetric = RMetric(problemR, norm_rp, pf=pareto_front)
    # normalize solutions before sending r-metric
    NUMS_transformer = Normalizer().fit(int_nums.population.objectives)
    norm_NUMS = NUMS_transformer.transform(int_nums.population.objectives)
    NUMS_PLUS_transformer = Normalizer().fit(int_nums_plus.population.objectives)
    norm_NUMS_PLUS = NUMS_PLUS_transformer.transform(int_nums_plus.population.objectives)
    # R-metric calls for R_IGD and R_HV
    rigd_NUMS, rhv_NUMS = rmetric.do(norm_NUMS, others=norm_NUMS_PLUS)
    rigd_NUMS_PLUS, rhv_NUMS_PLUS = rmetric.do(norm_NUMS_PLUS, others=norm_NUMS)
    print(rigd_NUMS)
    print(rhv_NUMS)
    
    print(rigd_NUMS_PLUS)
    print(rhv_NUMS_PLUS)
 

0.5739812349137765
8.395123245609227
0.5668094626863485
8.608338970566953
0.5572105671759578
8.258389129064618
0.5549035058773785
8.293174910886426


/home/giomara/PPSN2022/desdeo-emo/desdeo_emo/utilities/ReferenceVectors.py:462: RuntimeWarning:

divide by zero encountered in double_scalars

/home/giomara/PPSN2022/desdeo-emo/desdeo_emo/utilities/ReferenceVectors.py:462: RuntimeWarning:

invalid value encountered in double_scalars



0.5538469738729274
8.027571178842503
0.5542696814841522
8.014369968111676


/home/giomara/PPSN2022/desdeo-emo/desdeo_emo/utilities/ReferenceVectors.py:462: RuntimeWarning:

divide by zero encountered in double_scalars

/home/giomara/PPSN2022/desdeo-emo/desdeo_emo/utilities/ReferenceVectors.py:462: RuntimeWarning:

invalid value encountered in double_scalars



0.5641829868379012
7.919699921082041
0.5533770927345047
8.023493975210286


In [18]:
# Decision phase
# After the learning phase the reference vector which has the maximum number of assigned solutions forms ROI
max_assigned_vector = gp.get_max_assigned_vector(base.assigned_vectors)
for i in range(D):

    
    # since composite front grows after each iteration this call should be done for each iteration
    base = baseADM(cf, reference_vectors)
    # generates the next reference point for the decision phase
    response = gp.generateRP4decision(base, max_assigned_vector[0])

    # run algorithms with the new reference point
    pref_int_nums[2].response = pd.DataFrame(
        [response], columns=pref_int_nums[2].content["dimensions_data"].columns
    )
    pref_int_nums_plus[2].response = pd.DataFrame(
        [response], columns=pref_int_nums_plus[2].content["dimensions_data"].columns
    )

    pref_int_nums, plot_int_nums = int_nums.iterate(pref_int_nums[2])
    pref_int_nums_plus, plot_int_nums_plus = int_nums_plus.iterate(pref_int_nums_plus[2])
    # extend composite front with newly obtained solutions
    cf = generate_composite_front(
        cf, int_nums.population.objectives, int_nums_plus.population.objectives)
    
    # R-metric calculation
    ref_point = response.reshape(1, n_obj)
    rp_transformer = Normalizer().fit(ref_point)
    norm_rp = rp_transformer.transform(ref_point)
    # for decision phase, delta is specified as 0.2
    rmetric = RMetric(problemR, norm_rp, delta=0.2, pf=pareto_front)
    # normalize solutions before sending r-metric
    NUMS_transformer = Normalizer().fit(int_nums.population.objectives)
    norm_NUMS = NUMS_transformer.transform(int_nums.population.objectives)
    NUMS_PLUS_transformer = Normalizer().fit(int_nums_plus.population.objectives)
    norm_NUMS_PLUS = NUMS_PLUS_transformer.transform(int_nums_plus.population.objectives)
    rigd_NUMS, rhv_NUMS = rmetric.do(norm_NUMS, others=norm_NUMS_PLUS)
    rigd_NUMS_PLUS, rhv_NUMS_PLUS = rmetric.do(norm_NUMS_PLUS, others=norm_NUMS)
    
    print(rigd_NUMS)
    print(rhv_NUMS)

    
    print(rigd_NUMS_PLUS)
    print(rhv_NUMS_PLUS)
    

0.7208623929314236
8.553597301432717
0.715448841039608
8.562101443799747
0.7324843571418819
8.315125058686885
0.7129454716690592
8.575823090923336


In [ ]:
rp = gp.generateRP4learning(base)
visualize_3D_front_rp(base.composite_front, rp)

In [ ]:
rp

In [ ]:
max_assigned_vector = gp.get_max_assigned_vector(base.assigned_vectors)
rp2 = gp.generateRP4decision(base, max_assigned_vector)
visualize_3D_front_rp(base.composite_front, rp2)

In [ ]:
rp2